In [2]:
# Execute when running on Google Colab
!pip install neograd

In [ ]:
# Execute when running locally
# import sys
# sys.path.insert(0, '../tests')
# import _setup

In [3]:
import neograd as ng
import numpy as np
from neograd.nn.loss import SoftmaxCE
from neograd.nn.optim import Adam
from neograd.autograd.utils import grad_check
from neograd.nn.utils import get_batches
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [4]:
X, y = load_digits(return_X_y=True) # load data
X_train, X_test, y_train, y_test = train_test_split(X, y) # split data into train and test set

In [5]:
num_train, num_test = X_train.shape[0], X_test.shape[0] # number of train and test examples
num_iter = 200 # number of iterations
batch_size = 200 # batch size in each iteration

In [6]:
# normalize inputs
X_train_norm = (X_train - np.mean(X_train, axis=1, keepdims=True))/np.std(X_train, axis=1, keepdims=True)
X_test_norm = (X_test - np.mean(X_test, axis=1, keepdims=True))/np.std(X_test, axis=1, keepdims=True)

In [7]:
# Convert normalized inputs and targets to Tensor
X_train = ng.tensor(X_train_norm[:num_train,:].reshape(num_train,8,8))
X_test = ng.tensor(X_test_norm[:num_test,:].reshape(num_test,8,8))
y_train = ng.tensor(np.eye(10)[y_train[:num_train]])
y_test = ng.tensor(y_test[:num_test])

In [8]:
# Define the model
class NN(ng.nn.Model):
  def __init__(self):
    self.conv = ng.nn.Sequential(
      ng.nn.Conv2D((3,3)),
      ng.nn.ReLU()
    )
    self.stack = ng.nn.Sequential(
        ng.nn.Linear(36,10)
    )

  def forward(self, inputs):
    conv_outputs = self.conv(inputs)
    conv_outputs_flattened = conv_outputs.reshape((inputs.shape[0], 36))
    return self.stack(conv_outputs_flattened)

In [9]:
# configure the model
model = NN()
loss_fn = SoftmaxCE(axis=1)
optim = Adam(model.get_params(), 5e-3)

In [10]:
# train the model
for iter in range(num_iter):
  for batch_input, batch_target in get_batches(X_train, y_train, batch_size):
    optim.zero_grad()
    outputs = model(batch_input)
    loss = loss_fn(outputs, batch_target)
    loss.backward()
    optim.step()
  if iter%50==0:
    print(f"iter {iter+1}/{num_iter}\nloss: {loss}\n")

iter 1/200
loss: Tensor( 11.544053028208827,
 requires_grad=True,
 grad_fn=None,
 shape=() )


iter 51/200
loss: Tensor( 0.45721614541777483,
 requires_grad=True,
 grad_fn=None,
 shape=() )


iter 101/200
loss: Tensor( 0.15749437587925777,
 requires_grad=True,
 grad_fn=None,
 shape=() )


iter 151/200
loss: Tensor( 0.08086855586332076,
 requires_grad=True,
 grad_fn=None,
 shape=() )




In [11]:
# evaluate the model
with model.eval():
  test_outputs = model(X_test)
  probs = ng.nn.Softmax(1)(test_outputs.data)
  preds = np.argmax(probs.data, axis=1)

In [12]:
# get report
report = classification_report(y_test.data.astype(int).flatten(), preds.flatten())
print(report)
accuracy = accuracy_score(y_test.data.astype(int).flatten(), preds.flatten())
print('Accuracy:', accuracy)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        43
           1       0.93      0.91      0.92        46
           2       0.93      0.91      0.92        44
           3       0.91      0.88      0.89        48
           4       0.94      0.90      0.92        50
           5       0.87      0.98      0.92        41
           6       0.98      0.96      0.97        48
           7       0.95      1.00      0.97        37
           8       0.78      0.89      0.83        44
           9       0.98      0.84      0.90        49

    accuracy                           0.92       450
   macro avg       0.92      0.93      0.92       450
weighted avg       0.93      0.92      0.92       450

Accuracy: 0.9222222222222223


In [13]:
# perform gradient check on the model
grad_check(model, X_train, y_train, loss_fn)

Gradient Check Distance: 1.968236460958225e-08
Gradient Check PASSED


1.968236460958225e-08